# Estimating the Background

In this workbook we will investigate different methods for estimating the background for a test SNR. We will trial a simple constant background method, and a more complicated estimation method. We will compare the fit results between the two methods.